# 基于MindNLP的IA3模型微调

## 环境配置

1. 配置python3.9环境


In [2]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [1]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

***注：以上代码执行完成后，需点击左上角或右上角将kernel更换为python-3.9.0***

2. 安装mindspore2.2.14，安装指南详见：[MindSpore安装](https://www.mindspore.cn/install)
3. 安装MindNLP及相关依赖，MindNLP官方仓详见：[MindNLP](https://github.com/mindspore-lab/mindnlp)

In [1]:
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/unified/x86_64/mindspore-2.2.14-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install tokenizers==0.15.2 -i https://pypi.tuna.tsinghua.edu.cn/simple
!wget https://repo.mindspore.cn/mindspore-lab/mindnlp/newest/any/mindnlp-0.3.2-py3-none-any.whl
!pip install mindnlp-0.3.2-py3-none-any.whl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 743.0/743.0 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 8.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 9.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 5.4 MB/s eta 0:00:0000:0100:01m
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.0 MB/s eta 0:00:00
 

***注：执行如上命令完成安装后，请点击上方的restart kernel图标重启kernel，再进行实验***

### 数据集下载与处理

1. 数据集加载

本次使用的是情感分析数据集Financial PhraseBank。
该数据集由从LexisNexis数据库中随机挑选的4845个英语句子组成，其中这些句子由16名具有金融和商业背景的人进行注释。

In [9]:
# download dataset
!wget https://hf-mirror.com/datasets/financial_phrasebank/resolve/main/data/FinancialPhraseBank-v1.0.zip
!unzip FinancialPhraseBank-v1.0.zip

--2024-07-02 01:51:07--  https://hf-mirror.com/datasets/financial_phrasebank/resolve/main/data/FinancialPhraseBank-v1.0.zip



Resolving proxy.modelarts.com (proxy.modelarts.com)... 192.168.6.3



Connecting to proxy.modelarts.com (proxy.modelarts.com)|192.168.6.3|:80... connected.



Proxy request sent, awaiting response... 307 Temporary Redirect



Location: /datasets/takala/financial_phrasebank/resolve/main/data/FinancialPhraseBank-v1.0.zip [following]



--2024-07-02 01:51:08--  https://hf-mirror.com/datasets/takala/financial_phrasebank/resolve/main/data/FinancialPhraseBank-v1.0.zip



Reusing existing connection to hf-mirror.com:443.



Proxy request sent, awaiting response... 302 Found



Location: https://cdn-lfs.hf-mirror.com/datasets/financial_phrasebank/0e1a06c4900fdae46091d031068601e3773ba067c7cecb5b0da1dcba5ce989a6?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27FinancialPhraseBank-v1.0.zip%3B+filename%3D%22FinancialPhraseBank-v1.0.zip%22%3B&response-content-t

2. 数据预处理

    原始数据格式([text]@[label])：
    ```
    According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .@neutral
    ```
    使用mt0-base tokenizer对text进行处理，并将label转为对应数字格式，"negative"——>"0","neutral"——>"1","positive"——>"2"

In [2]:
def process_dataset(source, batch_size=32, shuffle=False):

    column_names = ['input_ids', 'attention_mask','labels','text_labels']
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    
    dataset = dataset.batch(batch_size)

    return dataset

class MSDataset:
    def __init__(self, filepath,tokenizer,max_length):
        self.path = filepath
        self.sentences = []
        self.labels = []
        self.text_labels = []
        self._load()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def _load(self):
        label_mapping = {
            "negative": 0,
            "neutral": 1,
            "positive": 2
        }
        with open(self.path, encoding="iso-8859-1") as f:
            for line in f:
                sentence, label_text = line.strip().split("@")
                self.sentences.append(sentence)
                self.labels.append(label_mapping[label_text])
                self.text_labels.append(label_text)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        text_labels = self.text_labels[index]
        model_inputs = self.tokenizer(sentence, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="np")
        labels = self.tokenizer(text_labels, max_length=3, padding="max_length", truncation=True, return_tensors="np")
        labels = labels["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels,self.text_labels[index]

    def __len__(self):
        return len(self.sentences)

In [3]:
from mindnlp.transformers import BartTokenizer

tokenizer_name_or_path = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(tokenizer_name_or_path)
len(tokenizer)

/home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.804 seconds.
Prefix dict has been built successfully.
878kB [00:22, 39.9kB/s]
446kB [00:24, 18.9kB/s] 
1.29MB [00:18, 73.4kB/s]
1.68kB [00:00, 3.66MB/s]                 


50265

In [4]:
from mindspore.dataset import GeneratorDataset
max_length = 128
batch_size = 8
dataset = process_dataset(MSDataset("./FinancialPhraseBank-v1.0/Sentences_AllAgree.txt",tokenizer,max_length),batch_size=batch_size)
train_dataset, eval_dataset = dataset.split([0.9, 0.1])

In [5]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[8, 1, 128], dtype=Int64, value=
[[[    0, 15791,  6407 ...     1,     1,     1]],
 [[    0, 15791,  6407 ...     1,     1,     1]],
 [[    0,   347, 14631 ...     1,     1,     1]],
 ...
 [[    0, 24699, 37258 ...     1,     1,     1]],
 [[    0, 45038,   766 ...     1,     1,     1]],
 [[    0, 14721, 37189 ...     1,     1,     1]]]), 'attention_mask': Tensor(shape=[8, 1, 128], dtype=Int64, value=
[[[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 ...
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]]]), 'labels': Tensor(shape=[8, 1, 3], dtype=Int64, value=
[[[    0, 12516,     2]],
 [[    0, 12516,     2]],
 [[    0, 12516,     2]],
 ...
 [[    0, 12516,     2]],
 [[    0, 12516,     2]],
 [[    0, 12516,     2]]]), 'text_labels': Tensor(shape=[8], dtype=String, value= ['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
 'neutral', 'neutral'])}


### 模型构建

1. 构建MT0-Base模型，并配置采用IA3微调的模型

In [6]:
# creating model
from mindnlp.transformers import  AutoModelForSeq2SeqLM

model_name_or_path = "facebook/bart-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

100%|██████████| 532M/532M [00:37<00:00, 14.7MB/s] 


In [7]:
from mindnlp.peft import get_peft_model, IA3Config, TaskType

peft_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, feedforward_cells=[])
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
#加载模型并打印微调参数量，可以看到仅有不到0.0026%的参数参与了微调。

trainable params: 36,864 || all params: 139,457,280 || trainable%: 0.026433901478646365


2. 动态学习率

In [8]:
from mindspore import ops
from mindspore.nn.learning_rate_schedule import LearningRateSchedule

class LinearWithWarmUp(LearningRateSchedule):
    """
    Warmup-decay learning rate.
    """
    def __init__(self, learning_rate, num_warmup_steps, num_training_steps):
        super().__init__()
        self.learning_rate = learning_rate
        self.num_warmup_steps = num_warmup_steps
        self.num_training_steps = num_training_steps

    def construct(self, global_step):
        if global_step < self.num_warmup_steps:
            return global_step / float(max(1, self.num_warmup_steps)) * self.learning_rate
        return ops.maximum(
            0.0, (self.num_training_steps - global_step) / (max(1, self.num_training_steps - self.num_warmup_steps))
        ) * self.learning_rate

### 模型训练
先定义训练的epoch, lr和优化器等；
后按照如下步骤定义训练逻辑：

1. 构建正向计算函数
2. 函数变换，获取微分函数
3. 定义训练一个step的逻辑
4. 遍历训练数据集进行模型训练，同时每一个epoch后，打印loss

In [9]:
num_epochs = 3
warmup_steps = 100
learning_rate = 8e-3

num_training_steps = num_epochs * len(train_dataset)

In [10]:
# optimizer and lr scheduler
from mindspore import nn

lr_scheduler = LinearWithWarmUp(learning_rate=learning_rate, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=lr_scheduler)


In [11]:
# training and evaluation
from mindspore import Tensor
import mindspore
from tqdm import tqdm
import numpy as np

num_batches = len(train_dataset)
num_batches_eval = len(eval_dataset)
                       
def forward_fn(input_ids,attention_mask,labels ):
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        return output.loss, output.logits
    
grad_fn = mindspore.value_and_grad(
        forward_fn, None,optimizer.parameters, has_aux=True
    )
for epoch in range(num_epochs):
    model.set_train(True)
    total_loss, total_step = 0, 0
    correct = 0
    total = 0
    with tqdm(total=num_batches) as t:
        for step, (input_ids,attention_mask,labels,_) in enumerate(train_dataset):
            input_ids  = input_ids.squeeze(axis=1)
            labels  = labels.squeeze(axis=1)
            attention_mask = attention_mask.squeeze(axis=1)
            (loss, logits), grad = grad_fn(input_ids,attention_mask,labels)
            optimizer(grad)
            total_loss += loss.asnumpy()
            total_step += 1
            curr_loss = total_loss / total_step
            t.set_postfix({'train-loss': f'{curr_loss:.2f}'})
            t.update(1)
    model.set_train(False)
    eval_loss = 0
    total_step = 0
    eval_preds = []
    text_labels = []
    with tqdm(total=num_batches_eval) as t:
        for step, (input_ids,attention_mask,labels,text) in enumerate(eval_dataset):
            input_ids = input_ids.squeeze(axis=1)
            labels = labels.squeeze(axis=1)
            attention_mask = attention_mask.squeeze(axis=1)
            outputs = model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
            loss = outputs.loss
            eval_loss += loss.asnumpy()
            total_step += 1           
            eval_loss = total_loss / total_step
            eval_preds.extend(
                tokenizer.batch_decode(np.argmax(outputs.logits.asnumpy(), -1), skip_special_tokens=True)
            )
            text_str = str(text.asnumpy())
            text_str = text_str.replace("[", "").replace("]", "").replace(",", "").replace("'", "")
            labels = text_str.split(" ")
            text_labels.extend(labels)
            t.set_postfix({'eval-loss': f'{eval_loss:.2f}'})
            t.update(1)
    for pred, text_label in zip(eval_preds, text_labels):
        if pred.strip() == text_label.strip():
            correct += 1
        total += 1
    accuracy = correct / total * 100
    print(f"{accuracy=} % on the evaluation dataset")
    eval_epoch_loss = eval_loss / eval_dataset.get_dataset_size()
    eval_ppl = np.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / train_dataset.get_dataset_size()
    train_ppl = np.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  0%|          | 0/255 [00:00<?, ?it/s][ERROR] CORE(2918,7f440ea0b740,python):2024-07-02-16:46:46.966.006 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_2918/729418715.py]
[ERROR] CORE(2918,7f440ea0b740,python):2024-07-02-16:46:46.966.069 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_2918/729418715.py]
[ERROR] CORE(2918,7f440ea0b740,python):2024-07-02-16:46:46.966.433 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_2918/729418715.py]
[ERROR] CORE(2918,7f440ea0b740,python):2024-07-02-16:46:46.966.454 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_2918/729418715.py]
100%|██████████| 28/28 [00:04<00:00,  6.45it/s, eval-loss=12.20]


accuracy=68.75 % on the evaluation dataset
epoch=0: train_ppl=3.8187727713706163 train_epoch_loss=1.3399291069630315 eval_ppl=1.546228559363969 eval_epoch_loss=0.4358187784127207


100%|██████████| 28/28 [00:04<00:00,  6.18it/s, eval-loss=3.14]


accuracy=63.39285714285714 % on the evaluation dataset
epoch=1: train_ppl=1.4119350392928587 train_epoch_loss=0.34496113184620353 eval_ppl=1.1187369974392896 eval_epoch_loss=0.11220036813875242


100%|██████████| 28/28 [00:04<00:00,  6.34it/s, eval-loss=3.33]

accuracy=75.0 % on the evaluation dataset
epoch=2: train_ppl=1.441003220083706 train_epoch_loss=0.36533955163231085 eval_ppl=1.1261768222624253 eval_epoch_loss=0.11882855314571335


### 微调模型推理

1. 保存训练后的微调模型权重

In [12]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [13]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

148K	facebook/bart-base_IA3_SEQ_2_SEQ_LM/adapter_model.ckpt


2. 加载保存的模型权重

In [14]:
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

3. 在测试集上进行推理

In [15]:
import datasets
text_column = "sentence"

i = 13
def load_dataset(filepath):
    sentences = []
    labels = []
    with open(filepath, encoding="iso-8859-1") as f:
        for line in f:
            sentence, label = line.strip().split("@")
            sentences.append(sentence)
            labels.append(label)
    
    dataset = datasets.Dataset.from_dict({
        "sentence": sentences,
        "label": labels,
    })
    return dataset

dataset = load_dataset("./FinancialPhraseBank-v1.0/Sentences_AllAgree.txt")
dataset = dataset.train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

inputs = tokenizer(dataset["validation"][text_column][i], return_tensors="ms")
print(dataset["validation"][text_column][i])
print(inputs)

outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
print(outputs)
print(tokenizer.batch_decode(outputs ,skip_special_tokens=True))

The company moved to an operating profit of EUR10 .9 m versus an operating loss of EUR15 .3 m. It also turned to EPS of EUR0 .08 versus loss per share of EUR0 .04 .
{'input_ids': Tensor(shape=[1, 46], dtype=Int64, value=
[[   0,  133,  138 ... 3387,  479,    2]]), 'attention_mask': Tensor(shape=[1, 46], dtype=Int64, value=
[[1, 1, 1 ... 1, 1, 1]])}


/home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/mindnlp/transformers/generation/utils.py:1561: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://hf-mirror.com/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[[    2     0 22173     2]]
['positive']
